In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import statsmodels as sm
from sklearn.model_selection import cross_val_score, train_test_split


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.preprocessing import PowerTransformer
from itertools import combinations
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
import numpy as np

import seaborn as sns
import featuretools as ft



In [14]:
df = pd.read_csv("po2_data.csv")
# delete irrelevant features
df = df.drop(['subject#', 'test_time'], axis=1) 
# column_relocation = df['total_updrs']
# del df['total_updrs']
# df.insert(loc=18, column='total_updrs', value=column_relocation)
male_x = df.query('sex == 0')

male_x = male_x.drop('sex', axis=1)
female_x = df.query('sex == 1')
female_x = female_x.drop('sex', axis=1)

# Separate explanatory variables (x) from the response variable (y)
# x = df.iloc[:, : -2]

# remove multicollinearity (r squared decreased)
# x = x.drop(['subject#','shimmer(apq3)', 'shimmer(%)', 'shimmer(apq5)', 'shimmer(apq11)', 'hnr'], axis=1)
print(female_x)


      age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
12     68    0.00835     0.000043      0.00444       0.00431      0.01333   
13     68    0.01374     0.000082      0.00701       0.00654      0.02104   
14     68    0.01194     0.000068      0.00630       0.00800      0.01889   
15     68    0.00812     0.000044      0.00375       0.00410      0.01124   
16     68    0.00316     0.000013      0.00097       0.00120      0.00291   
...   ...        ...          ...          ...           ...          ...   
5768   68    0.00795     0.000043      0.00390       0.00400      0.01170   
5769   68    0.00751     0.000041      0.00380       0.00291      0.01140   
5770   68    0.01286     0.000069      0.00781       0.00668      0.02342   
5771   68    0.00442     0.000020      0.00196       0.00201      0.00589   
5772   68    0.00415     0.000018      0.00160       0.00210      0.00481   

      shimmer(%)  shimmer(abs)  shimmer(apq3)  shimmer(apq5)  shimmer(apq11

In [15]:
male_y_1 = male_x.iloc[:, -2]
male_y_2 = male_x.iloc[:, -1]
female_y_1 = female_x.iloc[:, -2]
female_y_2 = female_x.iloc[:, -1]

female_x_1 = female_x.iloc[:, :-2]
female_x_2 = female_x.iloc[:, :-2]
male_x_1 = male_x.iloc[:, :-2]
male_x_2 = male_x.iloc[:, :-1]

print(female_x_1)
print(female_x_2)

# Apply cross validation to get the optimal ratio of splitting the dataset
def cross_validation(X, y):
    # Define a range of test set sizes
    test_size_range = [0.1, 0.2, 0.3, 0.4, 0.5]

    # Initialize an empty list to store mean performance scores
    mean_scores = []

    for test_size in test_size_range:
        # Initialize an empty list to store cross-validation scores for each fold
        fold_scores = []

        # Perform k-fold cross-validation for each test set size
        k = 5  # Number of folds
        for _ in range(k):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=0)
            model = LinearRegression()
            model.fit(X_train, y_train)
            scores = model.score(X_test, y_test)  # You can use any appropriate performance metric here
            fold_scores.append(scores)

        # Calculate the mean score for this test set size
        mean_score = np.mean(fold_scores)
        mean_scores.append(mean_score)

    # Choose the best test set size based on the mean performance scores
    best_test_size = test_size_range[np.argmax(mean_scores)]
    print(best_test_size)
        
    return best_test_size


ratio_1 = cross_validation(female_x_1, female_y_1)
ratio_2 = cross_validation(female_x_2, female_y_2)


def test(x_1, x_2, y_1, y_2, ratio_1=0.4, ratio_2=0.4):
    x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(x_1, y_1, test_size=ratio_1, random_state=0)
    x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(x_2, y_2, test_size=ratio_2, random_state=0)


    # Build a linear regression model
    model_1 = LinearRegression()

    # Train (fit) the linear regression model using the training data
    model_1.fit(x_train_1, y_train_1)

    model_2 = LinearRegression()

    # Train (fit) the linear regression model using the training data
    model_2.fit(x_train_2, y_train_2)

    print(f"Intercept: {model_2.intercept_}")
    print(f"Coefficient: {model_2.coef_}")

    # Use linear regression to predict the values of (y) in the test set
    # based on the values of x in the test set
    y_pred_1 = model_1.predict(x_test_1)
    y_pred_2 = model_2.predict(x_test_2)

    # Optional: Show the predicted values of (y) next to the actual values of (y)
    df_pred_1 = pd.DataFrame({"Actual": y_test_1, "Predicted": y_pred_1})
    df_pred_2 = pd.DataFrame({"Actual": y_test_2, "Predicted": y_pred_2})
    print(x_train_2)
    print(df_pred_2)

    # Compute standard performance metrics of the linear regression:

    # Mean Absolute Error
    mae = metrics.mean_absolute_error(y_test_1, y_pred_1)
    # Mean Squared Error
    mse = metrics.mean_squared_error(y_test_1, y_pred_1)
    # Root Mean Square Error
    rmse =  math.sqrt(metrics.mean_squared_error(y_test_1, y_pred_1))
    # Normalised Root Mean Square Error
    y_max = y_test_1.max()
    y_min = y_test_1.min()
    rmse_norm = rmse / (y_max - y_min)
    # R-Squared
    r_2 = metrics.r2_score(y_test_1, y_pred_1)
    
    print("MAE: ", mae)
    print("MSE: ", mse)
    print("RMSE: ", rmse)
    print("RMSE (Normalised): ", rmse_norm)
    print("R_squared: ",r_2)

    
    mae = metrics.mean_absolute_error(y_test_2, y_pred_2)
    # Mean Squared Error
    mse = metrics.mean_squared_error(y_test_2, y_pred_2)
    # Root Mean Square Error
    rmse =  math.sqrt(metrics.mean_squared_error(y_test_2, y_pred_2))
    # Normalised Root Mean Square Error
    y_max = y_test_2.max()
    y_min = y_test_2.min()
    rmse_norm = rmse / (y_max - y_min)

    # R-Squared
    r_2 = metrics.r2_score(y_test_2, y_pred_2)

    print("MAE: ", mae)
    print("MSE: ", mse)
    print("RMSE: ", rmse)
    print("RMSE (Normalised): ", rmse_norm)
    print("R_squared: ",r_2)

    

# Evaluate the model
test(female_x_1, female_x_2, female_y_1, female_y_2, ratio_1, ratio_2)


      age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
12     68    0.00835     0.000043      0.00444       0.00431      0.01333   
13     68    0.01374     0.000082      0.00701       0.00654      0.02104   
14     68    0.01194     0.000068      0.00630       0.00800      0.01889   
15     68    0.00812     0.000044      0.00375       0.00410      0.01124   
16     68    0.00316     0.000013      0.00097       0.00120      0.00291   
...   ...        ...          ...          ...           ...          ...   
5768   68    0.00795     0.000043      0.00390       0.00400      0.01170   
5769   68    0.00751     0.000041      0.00380       0.00291      0.01140   
5770   68    0.01286     0.000069      0.00781       0.00668      0.02342   
5771   68    0.00442     0.000020      0.00196       0.00201      0.00589   
5772   68    0.00415     0.000018      0.00160       0.00210      0.00481   

      shimmer(%)  shimmer(abs)  shimmer(apq3)  shimmer(apq5)  shimmer(apq11

In [16]:
"""
APPLY Z-SCORE STANDARDISATION
"""
scaler = StandardScaler()

# Drop the previously added constant

# Apply z-score standardisation to all explanatory variables
std_x_1 = scaler.fit_transform(female_x_1.values)
std_x_2 = scaler.fit_transform(female_x_2.values)

# Restore the column names of each explanatory variable
std_x_1_df = pd.DataFrame(std_x_1, index=female_x_1.index, columns=female_x_1.columns)
std_x_2_df = pd.DataFrame(std_x_2, index=female_x_2.index, columns=female_x_2.columns)
ratio_1 = cross_validation(female_x_1, female_y_1)
ratio_2 = cross_validation(female_x_2, female_y_2)

test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)


0.5
0.1
Intercept: 27.55603863767412
Coefficient: [ 2.39729850e+00 -9.39425575e-01  8.44090620e-01 -2.72404280e+02
  1.84131041e-01  2.73928687e+02  6.31791526e+00 -8.29657528e+00
  1.09379529e+03 -3.26766408e+00  2.53755395e+00 -1.09512636e+03
 -2.18497043e+00 -5.96143006e+00  5.62421033e-01 -1.52494943e+00
  9.35044209e-01]
           age  jitter(%)  jitter(abs)  jitter(rap)  jitter(ppq5)  \
1776  0.918389   0.075774    -0.004533     0.148093      0.052492   
2558 -0.685746  -0.417216    -0.427194    -0.513971     -0.360847   
5363 -0.685746  -0.145356    -0.139362    -0.149713     -0.093075   
1865 -0.780107  -0.626639    -0.681626    -0.585346     -0.486646   
424   0.163502  -0.319659    -0.412459    -0.385989     -0.303339   
...        ...        ...          ...          ...           ...   
2579  0.163502  -0.434126    -0.590021    -0.437674     -0.403978   
3808 -0.780107  -0.535586    -0.584863    -0.489359     -0.418355   
5165 -0.780107   0.051059    -0.038916     0.231774

In [17]:
# Log transform
var = ['jitter(%)', 'shimmer(dda)']
for v in var:
    zero_or_negative_mask = std_x_1_df[v] <= 0
    std_x_1_df.loc[zero_or_negative_mask, v] = 1e-6  # Replace with a small positive value
    std_x_1_df[v + "_log"] = std_x_1_df[v].apply(np.log)
    std_x_1_df = std_x_1_df.drop([v], axis=1)

    zero_or_negative_mask = std_x_2_df[v] <= 0
    std_x_2_df.loc[zero_or_negative_mask, v] = 1e-6  # Replace with a small positive value
    std_x_2_df[v + "_log"] = std_x_2_df[v].apply(np.log)
    std_x_2_df = std_x_2_df.drop([v], axis=1)

ratio_1 = cross_validation(female_x_1, female_y_1)
ratio_2 = cross_validation(female_x_2, female_y_2)

# print(std_x_df)
test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)


0.5
0.1
Intercept: 29.178086452665184
Coefficient: [ 2.35880573e+00 -5.51900904e-02 -3.01259253e+02  2.39612554e-01
  3.01345010e+02  6.72260134e+00 -8.28900540e+00  2.65682913e-01
 -4.08819096e+00  2.77753325e+00 -2.11765530e+00 -6.09008539e+00
  4.11925210e-01 -1.77118025e+00  1.25627815e-01  4.17503056e-01
 -2.71994149e-01]
           age  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
1776  0.918389    -0.004533     0.148093      0.052492     0.147278   
2558 -0.685746    -0.427194    -0.513971     -0.360847    -0.514781   
5363 -0.685746    -0.139362    -0.149713     -0.093075    -0.149705   
1865 -0.780107    -0.681626    -0.585346     -0.486646    -0.586156   
424   0.163502    -0.412459    -0.385989     -0.303339    -0.385979   
...        ...          ...          ...           ...          ...   
2579  0.163502    -0.590021    -0.437674     -0.403978    -0.436844   
3808 -0.780107    -0.584863    -0.489359     -0.418355    -0.488529   
5165 -0.780107    -0.038916     

In [18]:
# Gaussian transformation
scaler = PowerTransformer()

# Apply the transformer to make all explanatory variables more Gaussian-looking
std_x_1 = scaler.fit_transform(std_x_1_df.values)
std_x_2 = scaler.fit_transform(std_x_2_df.values)

# Restore column names of explanatory variables
std_x_1_df = pd.DataFrame(std_x_1, index=std_x_1_df.index, columns=std_x_1_df.columns)
std_x_2_df = pd.DataFrame(std_x_2, index=std_x_2_df.index, columns=std_x_2_df.columns)


ratio_1 = cross_validation(female_x_1, female_y_1)
ratio_2 = cross_validation(female_x_2, female_y_2)
test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)

0.5
0.1
Intercept: 27.536972695199537
Coefficient: [  2.46475897  -7.46342405 -36.81306734   3.62122622  40.26147638
  -0.30195774  -5.41740767   8.44655173 -13.90774629  10.68750722
   2.62947001  -4.2585495    0.77098914  -0.36041316  -2.85260413
   1.05732596  -1.5334223 ]
           age  jitter(abs)  jitter(rap)  jitter(ppq5)  jitter(ddp)  \
1776  0.925882     0.634168     0.923558      0.969448     0.922271   
2558 -0.714072    -0.489478    -1.094946     -0.597078    -1.098979   
5363 -0.714072     0.357192     0.302496      0.584858     0.302477   
1865 -0.801488    -1.581230    -1.471870     -1.427204    -1.476296   
424   0.114422    -0.436741    -0.509132     -0.281576    -0.509096   
...        ...          ...          ...           ...          ...   
2579  0.114422    -1.147184    -0.732323     -0.859127    -0.728587   
3808 -0.801488    -1.124163    -0.973533     -0.951562    -0.969483   
5165 -0.801488     0.570014     1.041570      0.826064     1.040459   
1749 -0.71407

In [19]:
# applying the forward stepwise selection, and the r square increase is neglecting 
def forward_stepwise_selection(X, y, ratio):
    X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, test_size=ratio, random_state=0)

    selected_features = []
    best_error = np.inf  # Initialize with a large value

    while len(selected_features) < len(X.columns):
        remaining_features = [feature for feature in X.columns if feature not in selected_features]
        errors = []
        
        for feature in remaining_features:
            current_features = selected_features + [feature]
            X_train_current = X_train_1[current_features]
            X_test_current = X_test_1[current_features]
            
            # Fit a model using statsmodels
            model = sm.OLS(y_train_1, sm.add_constant(X_train_current)).fit()
            
            # Make predictions and calculate error
            y_pred = model.predict(sm.add_constant(X_test_current))
            error = mean_squared_error(y_test_1, y_pred)
            errors.append((feature, error))
        
        # Find the feature that results in the lowest error
        best_feature, best_error_for_feature = min(errors, key=lambda x: x[1])
        
        # If the new feature improves the model, add it to the selected features
        if best_error_for_feature < best_error:
            selected_features.append(best_feature)
            best_error = best_error_for_feature
        else:
            break  # If no improvement, break the loop

    return selected_features

ratio_1 = cross_validation(std_x_1_df, female_y_1)
ratio_2 = cross_validation(std_x_2_df, female_y_2)

selected_features_1 = forward_stepwise_selection(std_x_1_df, female_y_1, ratio_1)
selected_features_2 = forward_stepwise_selection(std_x_2_df, female_y_2, ratio_2)


keep_1 = selected_features_1
keep_2 = selected_features_2
print(keep_1)
print(keep_2)

# keep = ['age', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
# ['age', 'sex', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
std_x_1_df = std_x_1_df[[x for x in keep_1]]
std_x_2_df = std_x_2_df[[x for x in keep_2]]

ratio_1 = cross_validation(std_x_1_df, female_y_1)
ratio_2 = cross_validation(std_x_2_df, female_y_2)


test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)
# def best_subset_selection(X, y, model):
#     best_subset = []
#     best_score = float('inf')
    
#     for k in range(1, len(X.columns) + 1):
#         for subset in combinations(X.columns, k):
#             X_subset = X[list(subset)]
            
#             X_train, X_val, y_train, y_val = train_test_split(X_subset, y, test_size=0.4    , random_state=42)
#             model.fit(X_train, y_train)
#             y_pred = model.predict(X_val)
#             mse = mean_squared_error(y_val, y_pred)
            
#             if mse < best_score:
#                 best_score = mse
#                 best_subset = list(subset)
    
#     return best_subset

# best_model = None
# best_subset = None
# best_error = np.inf
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(std_x_1_df, female_y_1, test_size=ratio_1, random_state=0)
# X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(std_x_2_df, female_y_2, test_size=ratio_2, random_state=0)

# model = LinearRegression()
# best_features_1 = best_subset_selection(X_train_1, y_train_1, model)
# best_features_2 = best_subset_selection(X_train_2, y_train_2, model)



0.1
0.1
['jitter(ddp)', 'age', 'jitter(abs)', 'hnr', 'rpde', 'jitter(ppq5)', 'ppe', 'nhr', 'shimmer(abs)', 'shimmer(apq11)', 'shimmer(%)', 'jitter(rap)', 'shimmer(dda)_log']
['hnr', 'age', 'ppe', 'jitter(ddp)', 'jitter(abs)', 'jitter(ppq5)', 'nhr', 'shimmer(dda)_log', 'shimmer(apq11)', 'shimmer(abs)', 'rpde', 'jitter(rap)']
0.1
0.1
Intercept: 27.54709074577845
Coefficient: [ -3.91629984   2.71922669  -2.34628133  50.74603081  -8.19079803
   2.5762106    3.18265758  -1.4596826    6.12607506  -6.95769395
   0.8936167  -45.30197228]
           hnr       age       ppe  jitter(ddp)  jitter(abs)  jitter(ppq5)  \
1776 -0.229098  0.925882  0.370815     0.922271     0.634168      0.969448   
2558  0.939977 -0.714072  0.438866    -1.098979    -0.489478     -0.597078   
5363 -0.419955 -0.714072  0.670055     0.302477     0.357192      0.584858   
1865  1.039027 -0.801488 -1.655333    -1.476296    -1.581230     -1.427204   
424  -0.656965  0.114422  0.763728    -0.509096    -0.436741     -0.281576

In [20]:
# keep_1 = best_features_1
# keep_2 = best_features_2
# keep_1 = ['age', 'jitter(abs)', 'jitter(rap)', 'jitter(ppq5)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'nhr', 'hnr', 'rpde', 'ppe']
# keep_2 = ['age', 'jitter(abs)', 'jitter(ppq5)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'hnr', 'rpde', 'ppe', 'jitter(%)_log', 'shimmer(dda)_log']

# print(keep_1)
# print(keep_2)
# keep = ['age', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
# ['age', 'sex', 'test_time', 'jitter(%)', 'jitter(abs)', 'jitter(rap)', 'jitter(ddp)', 'shimmer(%)', 'shimmer(apq3)', 'shimmer(apq5)', 'shimmer(apq11)', 'shimmer(dda)', 'nhr', 'hnr', 'dfa']
# std_x_1_df = std_x_1_df[[x for x in keep_1]]
# std_x_2_df = std_x_2_df[[x for x in keep_2]]

ratio_1 = cross_validation(std_x_1_df, female_y_1)
ratio_2 = cross_validation(std_x_2_df, female_y_2)


test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)

0.1
0.1
Intercept: 27.54709074577845
Coefficient: [ -3.91629984   2.71922669  -2.34628133  50.74603081  -8.19079803
   2.5762106    3.18265758  -1.4596826    6.12607506  -6.95769395
   0.8936167  -45.30197228]
           hnr       age       ppe  jitter(ddp)  jitter(abs)  jitter(ppq5)  \
1776 -0.229098  0.925882  0.370815     0.922271     0.634168      0.969448   
2558  0.939977 -0.714072  0.438866    -1.098979    -0.489478     -0.597078   
5363 -0.419955 -0.714072  0.670055     0.302477     0.357192      0.584858   
1865  1.039027 -0.801488 -1.655333    -1.476296    -1.581230     -1.427204   
424  -0.656965  0.114422  0.763728    -0.509096    -0.436741     -0.281576   
...        ...       ...       ...          ...          ...           ...   
2579 -0.050270  0.114422  0.079221    -0.728587    -1.147184     -0.859127   
3808  0.834995 -0.801488 -1.434501    -0.969483    -1.124163     -0.951562   
5165 -0.348466 -0.801488  0.011933     1.040459     0.570014      0.826064   
1749  1.79

In [21]:
# print the p-value to delete variables with p-values higher than 0.05
std_x_1_df = sm.add_constant(std_x_1_df)
model_1 = sm.OLS(female_y_1,std_x_1_df).fit()
pred = model_1.predict(std_x_1_df)
model_details_1 = model_1.summary()
print(model_details_1)

std_x_2_df = sm.add_constant(std_x_2_df)
model_2 = sm.OLS(female_y_2,std_x_2_df).fit()
pred = model_2.predict(std_x_2_df)
model_details_2 = model_2.summary()
print(model_details_2)



                            OLS Regression Results                            
Dep. Variable:            motor_updrs   R-squared:                       0.322
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     67.75
Date:                Thu, 05 Oct 2023   Prob (F-statistic):          4.90e-146
Time:                        10:36:54   Log-Likelihood:                -6304.7
No. Observations:                1867   AIC:                         1.264e+04
Df Residuals:                    1853   BIC:                         1.271e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               20.9246      0.165  

In [22]:
# After removing high-p value, the r squared increased
# std_x_1_df = std_x_1_df.drop(['jitter(rap)', 'jitter(ddp)'], axis=1)
# std_x_2_df = std_x_2_df.drop(['shimmer(apq11)'], axis=1)

ratio_1 = cross_validation(std_x_1_df, female_y_1)
ratio_2 = cross_validation(std_x_2_df, female_y_2)


test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)

0.1
0.1
Intercept: 27.54709074577845
Coefficient: [  0.          -3.91629984   2.71922669  -2.34628133  50.74603081
  -8.19079803   2.5762106    3.18265758  -1.4596826    6.12607506
  -6.95769395   0.8936167  -45.30197228]
      const       hnr       age       ppe  jitter(ddp)  jitter(abs)  \
1776    1.0 -0.229098  0.925882  0.370815     0.922271     0.634168   
2558    1.0  0.939977 -0.714072  0.438866    -1.098979    -0.489478   
5363    1.0 -0.419955 -0.714072  0.670055     0.302477     0.357192   
1865    1.0  1.039027 -0.801488 -1.655333    -1.476296    -1.581230   
424     1.0 -0.656965  0.114422  0.763728    -0.509096    -0.436741   
...     ...       ...       ...       ...          ...          ...   
2579    1.0 -0.050270  0.114422  0.079221    -0.728587    -1.147184   
3808    1.0  0.834995 -0.801488 -1.434501    -0.969483    -1.124163   
5165    1.0 -0.348466 -0.801488  0.011933     1.040459     0.570014   
1749    1.0  1.799300 -0.714072 -1.440323    -0.985404    -1.267931

In [23]:
# feature enigneering
std_x_1_df['doshn'] = std_x_1_df['hnr']**2 - std_x_1_df['nhr']
std_x_2_df['doshn'] = std_x_2_df['hnr']**2 - std_x_2_df['nhr']
std_x_1_df['dshn'] = std_x_1_df['hnr']**2 / std_x_1_df['nhr']
std_x_2_df['dshn'] = std_x_2_df['hnr']**2 / std_x_2_df['nhr']



# std_x_1_df['rsj'] = std_x_1_df['shimmer(%)'] / std_x_1_df['jitter(%)']
# std_x_2_df['doshn'] = std_x_2_df['shimmer(%)']**2 - std_x_2_df['jitter(%)']
# print(std_x_1_df.head())
ratio_1 = cross_validation(std_x_1_df, female_y_1)
ratio_2 = cross_validation(std_x_2_df, female_y_2)
test(std_x_1_df, std_x_2_df, female_y_1, female_y_2, ratio_1, ratio_2)


0.1
0.1
Intercept: 28.505369785916134
Coefficient: [ 0.00000000e+00 -2.30938905e+00  2.43155768e+00 -2.69025378e+00
  3.54790041e+01 -8.44863073e+00  3.62102861e+00  3.42140814e+00
 -8.20927165e-01  7.18248436e+00 -7.91329639e+00  9.93938057e-01
 -3.04936683e+01 -9.59157221e-01  2.34297753e-04]
      const       hnr       age       ppe  jitter(ddp)  jitter(abs)  \
1776    1.0 -0.229098  0.925882  0.370815     0.922271     0.634168   
2558    1.0  0.939977 -0.714072  0.438866    -1.098979    -0.489478   
5363    1.0 -0.419955 -0.714072  0.670055     0.302477     0.357192   
1865    1.0  1.039027 -0.801488 -1.655333    -1.476296    -1.581230   
424     1.0 -0.656965  0.114422  0.763728    -0.509096    -0.436741   
...     ...       ...       ...       ...          ...          ...   
2579    1.0 -0.050270  0.114422  0.079221    -0.728587    -1.147184   
3808    1.0  0.834995 -0.801488 -1.434501    -0.969483    -1.124163   
5165    1.0 -0.348466 -0.801488  0.011933     1.040459     0.5700

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1867 entries, 12 to 5772
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   const             1867 non-null   float64
 1   jitter(ddp)       1867 non-null   float64
 2   age               1867 non-null   float64
 3   jitter(abs)       1867 non-null   float64
 4   hnr               1867 non-null   float64
 5   rpde              1867 non-null   float64
 6   jitter(ppq5)      1867 non-null   float64
 7   ppe               1867 non-null   float64
 8   nhr               1867 non-null   float64
 9   shimmer(abs)      1867 non-null   float64
 10  shimmer(apq11)    1867 non-null   float64
 11  shimmer(%)        1867 non-null   float64
 12  jitter(rap)       1867 non-null   float64
 13  shimmer(dda)_log  1867 non-null   float64
 14  doshn             1867 non-null   float64
 15  dshn              1867 non-null   float64
dtypes: float64(16)
memory usage: 248.0 KB
Non